In [10]:
import pycatch22
import os
import scipy
import numpy as np
from sklearn.impute import SimpleImputer
import pandas as pd

In [11]:
training_mat_files = []
testing_mat_files = []
val_mat_files = []


test_data = os.path.join(os.getcwd(),'data/test_data')
training_data = os.path.join(os.getcwd(),'data/train_data')
val_data = os.path.join(os.getcwd(),'data/val_data')

for root, dirs, files in os.walk(test_data):
    for file in files:
        if file.endswith('.mat'):
            testing_mat_files.append(os.path.join(root, file))

for root, dirs, files in os.walk(training_data):
    for file in files:
        if file.endswith('.mat'):
            training_mat_files.append(os.path.join(root, file))

for root, dirs, files in os.walk(val_data):
    for file in files:
        if file.endswith('.mat'):
            val_mat_files.append(os.path.join(root, file))

            
mapping_df = pd.read_csv(os.path.join(os.getcwd(),'data/files_and_conditions1.csv'))
mapping_dict = dict(zip(mapping_df['Name'], mapping_df['Condition']))

In [13]:
# Assuming testing_mat_files and mapping_dict are defined
num_leads = 12

for i in range(num_leads):
    features = []
    labels = []

    # Load data from each file and compute features
    for file in training_mat_files:
        # Load the .mat file
        mat_data = scipy.io.loadmat(file)
        data_matrix = mat_data['my_matrix']

        # Compute features for the i-th lead
        results = pycatch22.catch22_all(data_matrix[i])

        # Append results and corresponding label
        features.append(results['values'])
        file_name = os.path.basename(file).split('.')[0]
        labels.append(mapping_dict.get(file_name, 'Unknown'))

    # Convert to NumPy arrays
    features_array = np.array(features)
    labels_array = np.array(labels)

    # Create a DataFrame and save to CSV
    df = pd.DataFrame(features_array)
    df['label'] = labels_array
    df.to_csv(f'training-hctsa-catch22-features-lead-{i}.csv', index=False, header=False)

print("Feature extraction and saving completed!")


Feature extraction and saving completed!


In [4]:
features = []
labels = []

for file in training_mat_files:
    mat_data = scipy.io.loadmat(file)
    data_matrix = mat_data['my_matrix']

    results = pycatch22.catch22_all(data_matrix[0])

    features.append(results['values'])

    file_name = os.path.basename(file).split('.')[0]
    labels.append(mapping_dict.get(file_name, 'Unknown'))

features = np.array(features)
labels = np.array(labels)

df = pd.DataFrame(features)
df['label'] = labels

df.to_csv('hctsa-catch22-features-training.csv', index=False, header=False)